In [1]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers, Model
from keras.regularizers import l2
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.utils import class_weight
import datetime

In [2]:
#model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = VGG16(include_top=False, pooling='max')

#last_layer = model.get_layer('fc1')
#print('last layer output shape:', last_layer.output_shape)
#last_output = last_layer.output

FEATURE_SIZE = 512
reduction = 512

In [3]:
#for layer in model.layers:
#    layer._name = layer._name + str('_C')

# Flatten the output layer to 1 dimension
#x = layers.Flatten()(last_output)
#cut_model = Model(model.input, x)

In [4]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(244, 244))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(model.predict(x))
    return features

In [5]:
%%time
print(get_features('00000').shape)


(1, 512)
Wall time: 2.28 s


In [6]:
%%time
data = np.zeros((10000,512))
id = np.array(range(0,10000))

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,:] = features
        if i % 100 == True:
            print(i)
    return data

#data = features()

Wall time: 990 µs


In [7]:
#save_ass = 'features_vgg16.csv'

In [8]:
#print(np.count_nonzero(data))
save_ass = 'features_vgg16_poolavg.csv'

#np.savetxt(save_ass, data, delimiter=',')

using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [9]:
#lets prepare the data for the NN

In [10]:
%%time

ABC1 = np.loadtxt("train_triplets.txt").astype('int') 

data = np.genfromtxt(save_ass, delimiter= ',')
#data = np.array(data[:,1:])

Wall time: 5.29 s


In [11]:
#ABC1 = np.loadtxt("train_triplets.txt").astype('int')

#svd = TruncatedSVD(n_components=reduction)
#svd.fit(data)
#data = svd.transform(data)
#print(data.shape)
#np.savetxt("transformed_1024_ResV2.csv", data, delimiter=',')

def load_features(int_id):
    return data[int_id]

ABC1 = np.loadtxt("train_triplets.txt").astype('int')

data = np.genfromtxt("transformed_2k_vgg16.csv", delimiter= ',')
data = np.array(data)
#data = np.array(data[:,1:])

def load_features(int_id):
    return data[int_id]

data2 = np.genfromtxt("transformed_2048_svd_arc_vgg16.csv", delimiter= ',')
data3 = np.genfromtxt("transformed_2048_svd_rand_vgg16.csv", delimiter= ',')

print(np.max(data2))
print(np.min(data2))
print(np.count_nonzero(data2>0)/10000)
print(data2)

print(np.max(data3))
print(np.min(data3))
print(np.count_nonzero(data3>0)/10000)
print(data3)

In [12]:
%%time

# There are 4 different permutations of ABC that we can know the answer of. (CAB and CBA we don't know)
ONE_SET_SIZE = 59515

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((ONE_SET_SIZE * 2,1))
data_y0 = np.zeros((ONE_SET_SIZE * 2,1))

#create the inverse dataset and a set where B is connected to A (and inv)

ACB0 = np.zeros((ONE_SET_SIZE,3)).astype('int')    # One needs 3 different assignments since the arrays would be linked otherwise.
BAC1 = np.zeros((ONE_SET_SIZE,3)).astype('int')
BCA0 = np.zeros((ONE_SET_SIZE,3)).astype('int')

ACB0[:,0] = ABC1[:,0]
BAC1[:,1] = ABC1[:,0]
BCA0[:,2] = ABC1[:,0]

ACB0[:,2] = ABC1[:,1]
BAC1[:,0] = ABC1[:,1]
BCA0[:,0] = ABC1[:,1]

ACB0[:,1] = ABC1[:,2]
BAC1[:,2] = ABC1[:,2]
BCA0[:,1] = ABC1[:,2]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((ABC1, BAC1, ACB0, BCA0),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((ONE_SET_SIZE * 4,FEATURE_SIZE * 3))

for i in range(0,3):
        for j in range(0,ONE_SET_SIZE * 4):
            data_nn_x[j,range(i*FEATURE_SIZE, FEATURE_SIZE*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%50000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

we're at entry: 0 0
we're at entry: 0 50000
we're at entry: 0 100000
we're at entry: 0 150000
we're at entry: 0 200000
we're at entry: 1 0
we're at entry: 1 50000
we're at entry: 1 100000
we're at entry: 1 150000
we're at entry: 1 200000
we're at entry: 2 0
we're at entry: 2 50000
we're at entry: 2 100000
we're at entry: 2 150000
we're at entry: 2 200000
Wall time: 29.7 s


In [13]:
#np.savetxt("data_nn_x.csv", data_nn_x, delimiter=',')
#data_nn_x.astype('float16')

In [14]:
%%time
collab = np.concatenate((data_nn_x,labels), axis=1) #dtype='float16'
np.random.shuffle(collab)

data_x = collab[:,:FEATURE_SIZE * 3]
data_y = collab[:,FEATURE_SIZE * 3]

Wall time: 1.55 s


In [15]:
#print("%d bytes" % (data_nn_x.size * data_nn_x.itemsize))
#np.savetxt("nn_data.csv",data_nn_x,delimiter=',')

In [16]:
%xdel data_nn_x
print(data_x.dtype)
print(data_y.dtype)

print(np.max(data_x))
print(np.min(data_x))

float64
float64
171.44268798828125
0.0


In [20]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when val_loss is no longer improving
        monitor="loss",
        min_delta=0,
        # Take the minimum of val_loss as a guideline.
        mode="max",
        # No longer improving" being further defined as "for at least 50 epochs.
        patience=300,
        # Since we have such a large paitience we restore back to the best weights after a long waittime.
        restore_best_weights = True,
        verbose = 1,
    )
]

In [22]:
%%time
#init=uniform
#TAKE IT Branda:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(1536, activation='relu', input_dim=FEATURE_SIZE * 3, kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.5))
#neuralNetwork.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.003)))
#neuralNetwork.add(BatchNormalization())
#neuralNetwork.add(Dropout(0.5))
neuralNetwork.add(Dense(256, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.3))
neuralNetwork.add(Dense(64, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())

neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

neuralNetwork.summary()

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(data_x, data_y, epochs=256, batch_size=1024, verbose=2, validation_split = 0.2, callbacks=callbacks)

#evaluation of the network prediction
#predict_test_nn = neuralNetwork.predict(df_test_onehot_nn)
#predict_train_nn = neuralNetwork.predict(df_train_onehot_nn)
#print(predict_test_nn)
#print(predict_train_nn)
#predict_test_nn = (predict_test_nn >= 0.5)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1536)              2360832   
_________________________________________________________________
batch_normalization_6 (Batch (None, 1536)              6144      
_________________________________________________________________
dropout_6 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               393472    
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)               

Layers:                         F.      Epoch   Val_Acc
4096-4096-1024-256-64-1         2048    64      0.7962
4096-4096-1024-256-64-1         2048    128     0.8043
6144-4096-2048-1024-256-64-1    2048    64      0.7943
6144-4096-2048-1024-256-64-1    2048    256     0.8044
4096-4096-1024-256-64-1         2048    64      0.8329 #using 0.1 initial dropout (score was super bad)
4096-4096-1024-256-64-1         2048    64      0.7961 #0.5 Drop except for last 0.2
6144-1024-256-64-1              2048    128     0.8031 #check the SVD value and retry again
8162-256-64-1                   3072    256     0.732
4096-256-64-1                   1024    128     0.718
4096-256-64-1                   2048    64      0.804
4096-256-64-1                   3072    128     0.805
8192-256-64-1                   3072    64      0.805


Lets try to find out what dim-red. is needed: 1024-0.5-64-0.2-1 32e
Factor:         Val_Acc:
3072            0.725
2048            0.796 #wtf is this even.
1024            0.712


Using the Max-Pooling-Avg: VGG16
1024-256-64-1                   512     128     0.6990

Using the Max-Pooling-Avg: InceptionResNet V2
2048-512-64-1                   512     128     0.511

In [23]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,FEATURE_SIZE * 3))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*FEATURE_SIZE , FEATURE_SIZE*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = neuralNetwork.predict(test_nn_x)

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000


In [24]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_retrain_v14.txt",np.round(export,decimals=0), fmt='%i')